In [ ]:
x1 , y1 , z1= float(5.0),  float(0.0 ), float(0.0)
x2 , y2 , z2= float(-2.5), float(4.330127) , float(0.0)
x3 , y3 , z3= float(-2.5), float(-4.330127) , float(0.0)
xc , yc , zc= float(0.0) , float(0.0) , float(0.0)
                                           
xd1 , yd1 , zd1= float(5.0) , float(-5.0) , float(0.0)
xd2 , yd2 , zd2= float(0.0), float(10.0) , float(-10.0)
xd3 , yd3 , zd3= float(-10.0) , float(0.0) , float(10.0)
xdc , ydc , zdc= float(0.0) , float(0.0) , float(0.0)

In [99]:
n=3
dimensions = 3
R  = 0
m  = np.array([2,1,1])
Mc = 1e-5
k  = 20.91
x , x_dot , x_dot_dot = np.empty((4,3)) , np.empty((4,3)) , np.empty((4,3))
#omega    =
#omega_dot=
chaosFactor , chaosShiftChance = 10 , 0.25 



def dvdt(t, var):
    output = np.empty(len(np.ravel(y,'C')))
    
    for i in range(0,n):
        for j in range(0,dimensions):
            x_dot_dot[i,j]= -(k/m[i])*(x[3,j]-x[i,j])*( R/(np.sum((x[3]-x[i])**2)**0.5) - 1 )
            #omega_dot_odt[i,j] = (5*A[i,j]/(2*m[i]*Ra*Ra))
    #spring center
    for j in range(0,dimensions):
        x_dot_dot[3,j]=0
        for i in range(0,n):
            x_dot_dot[3,j]+= (k/Mc)*(x[3,j]-x[i,j])*(2*R/(np.sum((x[3]-x[i])**2)**0.5) - 1 )
    #Randomness
    #for i in range(0,n+1):
    #    x_dot_dot[i] +=A[i]*chaosFactor
    output[0:12] =  np.ravel(x_dot,'C') 
    output[12:24] = np.ravel(x_dot_dot,'C')
    return output


    

def runge_kutta(t0, y0, v0, t_end, dt, k, m,RL):
    t_values = np.arange(t0, t_end, dt)
    y_values = np.empty((len(t_values),12))
    v_values = np.empty((len(t_values),12))

    y_values[0] = y0
    v_values[0] = v0

    for i in range(1, len(t_values)):
        t = t_values[i-1]
        y = y_values[i-1]
        v = v_values[i-1]
        
        k1_y = dydt(t, y, v, k, m,RL)
        k1_v = dvdt(t, y, v, k, m,RL)

        k2_y = dydt(t + dt/2, y + k1_y*dt/2, v + k1_v*dt/2, k, m,RL)
        k2_v = dvdt(t + dt/2, y + k1_y*dt/2, v + k1_v*dt/2, k, m,RL)

        k3_y = dydt(t + dt/2, y + k2_y*dt/2, v + k2_v*dt/2, k, m,RL)
        k3_v = dvdt(t + dt/2, y + k2_y*dt/2, v + k2_v*dt/2, k, m,RL)

        k4_y = dydt(t + dt, y + k3_y*dt, v + k3_v*dt, k, m,RL)
        k4_v = dvdt(t + dt, y + k3_y*dt, v + k3_v*dt, k, m,RL)

        y_values[i] = y + (dt/6) * (k1_y + 2*k2_y + 2*k3_y + k4_y)
        v_values[i] = v + (dt/6) * (k1_v + 2*k2_v + 2*k3_v + k4_v)

    return t_values, y_values, v_values

    

IndentationError: expected an indented block after 'for' statement on line 34 (2429382414.py, line 37)

In [98]:
import numpy as np

def dydt(t, y, v, k, m,RL):
    #v[0:12] =[xd1 , yd1 , zd1 , xd2 , yd2 , zd2 , xd3 , yd3 , zd3 , xdc , ydc , zdc]
    return v

def dvdt(t, y, v, k, m,RL):
    #y[0:12]    =[x1 , y1 , z1 , x2 , y2 , z2 , x3 , y3 , z3 , xc , yc , zc]
    #Imposing limitation(this is the hard part):
    y[y>1000]=1000
    
    dvdt = np.empty_like(y)
    m1 , m2 , m3 , Mc = m[0] , m[1] , m[2] , m[3]
    pos1 , pos2, pos3 , posc= y[0:3] , y[3:6] , y[6:9] , y[9:12]
    
    dvdt[0:3]= -k*(y[9:12]-y[0:3])/m1 * ( -1  +RL/(np.sum((posc-pos1)**2)**0.5)  )
    dvdt[3:6]= -k*(y[9:12]-y[3:6])/m2 * ( -1  +RL/(np.sum((posc-pos2)**2)**0.5)  )
    dvdt[6:9]= -k*(y[9:12]-y[6:9])/m3 * ( -1  +RL/(np.sum((posc-pos3)**2)**0.5) )
    #dvdt[9:12]=0
    for j in range(0,3):
        dvdt[j+9]=  k*(y[j+9]-y[j])/Mc  *  (-1 + 2*RL/( np.sum((posc-pos1)**2)**0.5)  ) \
                    +k*(y[j+9]-y[j+3])/Mc* (-1 + 2*RL/( np.sum((posc-pos2)**2)**0.5)  ) \
                    +k*(y[j+9]-y[j+6])/Mc* (-1 + 2*RL/( np.sum((posc-pos3)**2)**0.5)  )
    
    #print( np.sum((posc-pos1)**2)**0.5 , np.sum((posc-pos2)**2)**0.5 ,np.sum((posc-pos3)**2)**0.5) 
    return dvdt

def runge_kutta(t0, y0, v0, t_end, dt, k, m,RL):
    t_values = np.arange(t0, t_end, dt)
    y_values = np.empty((len(t_values),12))
    v_values = np.empty((len(t_values),12))

    y_values[0] = y0
    v_values[0] = v0

    for i in range(1, len(t_values)):
        t = t_values[i-1]
        y = y_values[i-1]
        v = v_values[i-1]
        
        k1_y = dydt(t, y, v, k, m,RL)
        k1_v = dvdt(t, y, v, k, m,RL)

        k2_y = dydt(t + dt/2, y + k1_y*dt/2, v + k1_v*dt/2, k, m,RL)
        k2_v = dvdt(t + dt/2, y + k1_y*dt/2, v + k1_v*dt/2, k, m,RL)

        k3_y = dydt(t + dt/2, y + k2_y*dt/2, v + k2_v*dt/2, k, m,RL)
        k3_v = dvdt(t + dt/2, y + k2_y*dt/2, v + k2_v*dt/2, k, m,RL)

        k4_y = dydt(t + dt, y + k3_y*dt, v + k3_v*dt, k, m,RL)
        k4_v = dvdt(t + dt, y + k3_y*dt, v + k3_v*dt, k, m,RL)

        y_values[i] = y + (dt/6) * (k1_y + 2*k2_y + 2*k3_y + k4_y)
        v_values[i] = v + (dt/6) * (k1_v + 2*k2_v + 2*k3_v + k4_v)

    return t_values, y_values, v_values

t0 = 0     # Initial time
t_end = 100 # End time
dt = 0.001  # Time step
k = 20.91    # Constant k
m = np.array([2,1,1,1e-5])    # Constant m
RL = 0

x1 , y1 , z1= float(5.0),  float(0.0 ), float(0.0)
x2 , y2 , z2= float(-2.5), float(4.330127) , float(0.0)
x3 , y3 , z3= float(-2.5), float(-4.330127) , float(0.0)
xc , yc , zc= float(0.0) , float(0.0) , float(0.0)
                                           
xd1 , yd1 , zd1= float(5.0) , float(-5.0) , float(0.0)
xd2 , yd2 , zd2= float(0.0), float(10.0) , float(-10.0)
xd3 , yd3 , zd3= float(-10.0) , float(0.0) , float(10.0)
xdc , ydc , zdc= float(0.0) , float(0.0) , float(0.0)

y0 , v0 = np.empty(12) , np.empty(12)
y0[0:12]    =[x1 , y1 , z1 , x2 , y2 , z2 , x3 , y3 , z3 , xc , yc , zc]
v0[0:12]    =[xd1 , yd1 , zd1 , xd2 , yd2 , zd2 , xd3 , yd3 , zd3 , xdc , ydc , zdc]
#print( dvdt(t0, y0, v0, k, m,RL))
t_values, y_values, v_values = runge_kutta(t0, y0, v0, t_end, dt, k, m,RL)

np.savetxt('motion_data.txt', np.column_stack((t_values, y_values[:,0],y_values[:,1],
                                                  y_values[:,2] , y_values[:,3],
                                                  y_values[:,4], y_values[:,5], 
                                                  y_values[:,6]  , y_values[:,7],
                                                  y_values[:,8], y_values[:,9],
                                                  y_values[:,10], y_values[:,11]  ) 
                                                ) , delimiter=',' , 
header='Time (s), x1 , y1 , z1 , x2 , y2 , z2 , x3 , y3 , z3 , xc , yc , zc ')

In [103]:
a=y_values[t_values%1.00==0]
a[0:10]

array([[ 5.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        -2.50000000e+00,  4.33012700e+00,  0.00000000e+00,
        -2.50000000e+00, -4.33012700e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-2.60910882e+00,  7.47333531e-01,  0.00000000e+00,
         4.02630497e+00, -2.43283482e+00,  2.16558235e+00,
         6.19188732e+00,  9.38170249e-01, -2.16558235e+00,
         2.53636401e+00, -2.49110900e-01,  0.00000000e+00],
       [ 3.90460585e+00, -1.24028253e+00,  0.00000000e+00,
        -1.10317978e+00, -2.62063558e+00, -6.02855797e-01,
        -1.70603558e+00,  5.10119650e+00,  6.02855797e-01,
         3.65128199e-01,  4.13427051e-01,  0.00000000e+00],
       [ 7.03485799e-01,  1.31105230e+00,  0.00000000e+00,
         2.79538658e+00,  1.44925598e+00, -1.99775908e+00,
         7.97627500e-01, -4.07135621e+00,  1.99775908e+00,
         1.43216703e+00, -4.37016949e-01,  0.00000000e+00],
       [-4.97616617e-01, -9.35553560e-01,  0.0000000